# Convert existing PyTorch code to Ray AIR

If you already have working PyTorch code, you don't have to start from scratch to utilize the benefits of Ray Air. Instead, you can continue to use your existing code and incrementally add Ray AIR components as needed.

Some of the benefits you'll get by using Ray AIR with your existing PyTorch training code:

- Easy distributed data-parallel training on a cluster
- Automatic checkpointing/fault tolerance and result tracking
- Parallell data preprocessing
- Seamless integration with hyperparameter tuning
- Scalable batch prediction
- Scalable model serving

This tutorial will show you how to start with Ray AIR from your existing PyTorch training code. We will learn how to **distribute your training** and do **scalable batch prediction**.


## The example code

The example code we'll be using is that of the [PyTorch quickstart tutorial](https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html). This code trains a neural network classifier on the FashionMNIST dataset.

You can find the code we used for this tutorial [here on GitHub](https://github.com/pytorch/tutorials/blob/8dddccc4c69116ca724aa82bd5f4596ef7ad119c/beginner_source/basics/quickstart_tutorial.py).

## Unmodified
Let's start with the unmodified code from the example. A thorough explanation of the parts is given in the full tutorial - we'll just focus on the code here.

We start with some imports:

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

Then we download the data:

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


/Users/kai/.pyenv/versions/3.7.7/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


We can now define the dataloaders:

In [3]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

We can then define and instantiate the neural network:

In [4]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Define our optimizer and loss:

In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

And finally our training loop. Note that we renamed the function from `train` to `train_epoch` to avoid conflicts with the Ray Train module later (which is also called `train`):

In [6]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

And while we're at it, here is our validation loop (note that we sneaked in a `return test_loss` statement and also renamed the function):

In [7]:
def test_epoch(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

Now we can trigger training and save a model:

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_epoch(train_dataloader, model, loss_fn, optimizer)
    test_epoch(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.318363  [    0/60000]
loss: 2.296939  [ 6400/60000]
loss: 2.287377  [12800/60000]
loss: 2.267626  [19200/60000]
loss: 2.257751  [25600/60000]
loss: 2.232479  [32000/60000]
loss: 2.232957  [38400/60000]
loss: 2.205788  [44800/60000]
loss: 2.197205  [51200/60000]
loss: 2.167193  [57600/60000]
Test Error: 
 Accuracy: 49.4%, Avg loss: 2.170019 

Epoch 2
-------------------------------
loss: 2.184961  [    0/60000]
loss: 2.165998  [ 6400/60000]
loss: 2.121752  [12800/60000]
loss: 2.127611  [19200/60000]
loss: 2.083768  [25600/60000]
loss: 2.028539  [32000/60000]
loss: 2.048409  [38400/60000]
loss: 1.977064  [44800/60000]
loss: 1.968806  [51200/60000]
loss: 1.905046  [57600/60000]
Test Error: 
 Accuracy: 56.6%, Avg loss: 1.913422 

Epoch 3
-------------------------------
loss: 1.944019  [    0/60000]
loss: 1.907888  [ 6400/60000]
loss: 1.808830  [12800/60000]
loss: 1.841802  [19200/60000]
loss: 1.737142  [25600/60000]
loss: 1.688371  [32000/600

In [9]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


We'll cover the rest of the tutorial (loading the model and doing batch prediction) later!

## Introducing a wrapper function (no Ray AIR, yet!)
The notebook-style from the tutorial is great for tutorials, but in your production code you probably wrapped the actual training logic in a function. So let's do this here, too.

Note that we do not add or alter any code here (apart from variable definitions) - we just take the loose bits of code in the current tutorial and put them into one function.

In [10]:
def train_func():
    batch_size = 64
    lr = 1e-3
    epochs = 5
    
    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)
    
    # Get cpu or gpu device for training.
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using {device} device")
    
    model = NeuralNetwork().to(device)
    print(model)
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_epoch(train_dataloader, model, loss_fn, optimizer)
        test_epoch(test_dataloader, model, loss_fn)

    print("Done!")

Let's see it in action again:

In [11]:
train_func()

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.303741  [    0/60000]
loss: 2.288284  [ 6400/60000]
loss: 2.272002  [12800/60000]
loss: 2.263862  [19200/60000]
loss: 2.247184  [25600/60000]
loss: 2.224714  [32000/60000]
loss: 2.226805  [38400/60000]
loss: 2.198349  [44800/60000]
loss: 2.193284  [51200/60000]
loss: 2.161947  [57600/60000]
Test Error: 
 Accuracy: 53.0%, Avg loss: 2.155591 

Epoch 2
-------------------------------
loss: 2.167714  [    0/60000]
loss: 2.154427  [ 6400/60000]
loss: 2.105020  [12800/60000]
loss: 2.117083  [19200/60000]
loss: 2.066656  [25600/60000]
loss: 2.015281  [32000/60000]
loss: 2.029835  [38400/60000]
loss: 1.961622  [44800

The output should look very similar to the previous ouput.

## Starting with Ray AIR: Distribute the training

As a first step, we want to distribute the training across multiple workers. For this we want to

1. Use data-parallel training by sharding the training data
2. Setup the model to communicate gradient updates across machines
3. Report the results back to Ray Train.


To facilitate this, we only need a few changes to the code:

1. We import Ray Train:

```python
import ray.train as train
```


2. We use a `config` dict to configure some hyperparameters (this is not strictly needed but good practice):

```python
def train_func(config: dict):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["epochs"]
```

3. We dynamically adjust the worker batch size according to the number of workers:

```python
    batch_size_per_worker = batch_size // train.world_size()
```

4. We prepare the data loader for distributed data sharding:

```python
    train_dataloader = train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = train.torch.prepare_data_loader(test_dataloader)
```

5. We prepare the model for distributed gradient updates:

```python
    model = train.torch.prepare_model(model)
```

Note that `train.torch.prepare_model()` also automatically takes care of setting up devices (e.g. GPU training) - so we can get rid of those lines in our current code!


6. We capture the validation loss and report it to Ray train:

```python
        test_loss = test(test_dataloader, model, loss_fn)
        train.report(loss=test_loss)
```

7. In the `train_epoch()` and `test_epoch()` functions we divide the `size` by the world size:

```python
    size = len(dataloader.dataset) // train.world_size()  # Divide by word size
```

8. In the `train_epoch()` function we can get rid of the device mapping. Ray Train does this for us:

```python
        # We don't need this anymore! Ray Train does this automatically:
        # X, y = X.to(device), y.to(device) 
```

That's it - you need less than 10 lines of Ray Train-specific code and can otherwise continue to use your original code.

Let's take a look at the resulting code. First the `train_epoch()` function (2 lines changed, and we also commented out the print statement):

In [12]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) // train.world_size()  # Divide by word size
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # We don't need this anymore! Ray Train does this automatically:
        # X, y = X.to(device), y.to(device)  

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

Then the `test_epoch()` function (1 line changed, and we also commented out the print statement):

In [13]:
def test_epoch(dataloader, model, loss_fn):
    size = len(dataloader.dataset) // train.world_size()  # Divide by word size
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

And lastly, the wrapping `train_func()` where we added 4 lines and modified 2 (apart from the config dict):

In [14]:
import ray.train as train


def train_func(config: dict):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["epochs"]
    
    batch_size_per_worker = batch_size // train.world_size()
    
    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size_per_worker)
    test_dataloader = DataLoader(test_data, batch_size=batch_size_per_worker)
    
    train_dataloader = train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = train.torch.prepare_data_loader(test_dataloader)
    
    model = NeuralNetwork()
    model = train.torch.prepare_model(model)
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    for t in range(epochs):
        train_epoch(train_dataloader, model, loss_fn, optimizer)
        test_loss = test_epoch(test_dataloader, model, loss_fn)
        train.report(loss=test_loss)

    print("Done!")

Now we'll use Ray Train's TorchTrainer to kick off the training. Note that we can set the hyperparmameters here! In the `scaling_config` we can also configure how many parallel workers to use and if we want to enable GPU training or not.

In [15]:
from ray.train.torch import TorchTrainer


trainer = TorchTrainer(
    train_loop_per_worker=train_func,
    train_loop_config={"lr": 1e-3, "batch_size": 64, "epochs": 4},
    scaling_config={"num_workers": 2, "use_gpu": False},
)
result = trainer.fit()
print(f"Last result: {result.metrics}")

2022-06-13 16:33:13,925	INFO services.py:1483 -- View the Ray dashboard at http://127.0.0.1:8265


Trial name,status,loc,iter,total time (s),loss,_timestamp,_time_this_iter_s
TorchTrainer_c3ccf_00000,TERMINATED,127.0.0.1:74854,4,36.6774,1.23507,1655130840,8.12762


2022-06-13 16:33:20,337	WARNING worker.py:1625 -- Warning: The actor TrainTrainable is very large (52 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(BaseWorkerMixin pid=74861) 2022-06-13 16:33:26,270	INFO config.py:71 -- Setting up process group for: env:// [rank=1, world_size=2]
(BaseWorkerMixin pid=74860) 2022-06-13 16:33:26,271	INFO config.py:71 -- Setting up process group for: env:// [rank=0, world_size=2]
(BaseWorkerMixin pid=74861) 2022-06-13 16:33:27,678	INFO train_loop_utils.py:293 -- Moving model to device: cpu
(BaseWorkerMixin pid=74861) 2022-06-13 16:33:27,678	INFO train_loop_utils.py:331 -- Wrapping provided model in DDP.
(BaseWorkerMixin pid=74860) 2022-06-13 16:33:27,678	INFO train_loop_utils.py:293 -- Moving model to device: cpu
(BaseWorkerMixin pid=74860) 2022-06-13 16:33:27,678	INFO train_loop_utils.py:331 -- Wrapping provided model in DDP.


Result for TorchTrainer_c3ccf_00000:
  _time_this_iter_s: 8.313539028167725
  _timestamp: 1655130815
  _training_iteration: 1
  date: 2022-06-13_16-33-35
  done: false
  experiment_id: ce8d09ec263c45ca9e44a3076977f2d8
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 1
  loss: 2.1526706203533585
  node_ip: 127.0.0.1
  pid: 74854
  time_since_restore: 11.971886157989502
  time_this_iter_s: 11.971886157989502
  time_total_s: 11.971886157989502
  timestamp: 1655130815
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: c3ccf_00000
  warmup_time: 0.0035440921783447266
  
Result for TorchTrainer_c3ccf_00000:
  _time_this_iter_s: 8.312272787094116
  _timestamp: 1655130824
  _training_iteration: 2
  date: 2022-06-13_16-33-44
  done: false
  experiment_id: ce8d09ec263c45ca9e44a3076977f2d8
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 2
  loss: 1.8812954858609825
  node_ip: 127.0.0.1
  pid: 74854
  time_since_restore: 20.284203052520752
  time_this_ite

2022-06-13 16:34:01,652	INFO tune.py:742 -- Total run time: 45.12 seconds (43.74 seconds for the tuning loop).


Last result: {'loss': 1.235072280191312, '_timestamp': 1655130840, '_time_this_iter_s': 8.127615928649902, '_training_iteration': 4, 'time_this_iter_s': 8.127733945846558, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 4, 'trial_id': 'c3ccf_00000', 'experiment_id': 'ce8d09ec263c45ca9e44a3076977f2d8', 'date': '2022-06-13_16-34-00', 'timestamp': 1655130840, 'time_total_s': 36.6774001121521, 'pid': 74854, 'hostname': 'Kais-MacBook-Pro.local', 'node_ip': '127.0.0.1', 'config': {}, 'time_since_restore': 36.6774001121521, 'timesteps_since_restore': 0, 'iterations_since_restore': 4, 'warmup_time': 0.0035440921783447266, 'experiment_tag': '0'}


Great, this works! You're now training your model in parallel. You could now scale this up to more nodes and workers on your Ray cluster.

But there are a few improvements we can make to the code in order to get the most of the system. For one, we should enable **checkpointing** to get access to the trained model afterwards. Additionally, we should optimize the **data loading** to take place within the workers.

### Enabling checkpointing to retrieve the model
Enabling checkpointing is pretty easy - we just need to call the `train.save_checkpoint()` API and pass the model state to it:

```python
    train.save_checkpoint(epoch=t, model=model.module.state_dict())
```

Note that the `model.module` part is needed because the model gets wrapped in `torch.nn.DistributedDataParallel`.

### Move the data loader to the training function

You may have noticed a warning: `Warning: The actor TrainTrainable is very large (52 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.`.

This is because we load the data outside the training function. Ray then serializes it to make it accessible to the remote tasks (that may get executed on a remote node!). This is not too bad with just 52 MB of data, but imagine this were a full image dataset - you wouldn't want to ship this around the cluster unnecessarily. Instead, you should move the dataset loading part into the `train_func()`. This will then download the data *to disk* once per machine and result in much more efficient data loading.

The result looks like this:

In [16]:
def load_data():
    # Download training data from open datasets.
    training_data = datasets.FashionMNIST(
        root="data",
        train=True,
        download=True,
        transform=ToTensor(),
    )

    # Download test data from open datasets.
    test_data = datasets.FashionMNIST(
        root="data",
        train=False,
        download=True,
        transform=ToTensor(),
    )
    return training_data, test_data


def train_func(config: dict):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["epochs"]
    
    batch_size_per_worker = batch_size // train.world_size()
    
    training_data, test_data = load_data()  # <- this is new!
    
    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size_per_worker)
    test_dataloader = DataLoader(test_data, batch_size=batch_size_per_worker)
    
    train_dataloader = train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = train.torch.prepare_data_loader(test_dataloader)
    
    model = NeuralNetwork()
    model = train.torch.prepare_model(model)
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    
    for t in range(epochs):
        train_epoch(train_dataloader, model, loss_fn, optimizer)
        test_loss = test_epoch(test_dataloader, model, loss_fn)
        train.save_checkpoint(epoch=t, model=model.module.state_dict())  # <- this is new!
        train.report(loss=test_loss)

    print("Done!")

Let's train again:

In [17]:
trainer = TorchTrainer(
    train_loop_per_worker=train_func,
    train_loop_config={"lr": 1e-3, "batch_size": 64, "epochs": 4},
    scaling_config={"num_workers": 2, "use_gpu": False},
)
result = trainer.fit()


Trial name,status,loc,iter,total time (s),loss,_timestamp,_time_this_iter_s
TorchTrainer_de45a_00000,TERMINATED,127.0.0.1:74893,4,34.6658,1.25396,1655130879,7.7791


(BaseWorkerMixin pid=74906) 2022-06-13 16:34:06,675	INFO config.py:71 -- Setting up process group for: env:// [rank=0, world_size=2]
(BaseWorkerMixin pid=74907) 2022-06-13 16:34:06,675	INFO config.py:71 -- Setting up process group for: env:// [rank=1, world_size=2]
(BaseWorkerMixin pid=74906) /Users/kai/.pyenv/versions/3.7.7/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
(BaseWorkerMixin pid=74906)   return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
(BaseWorkerMixin pid=74906) 2022-06-13 16:34:07,831	INFO train_l

Result for TorchTrainer_de45a_00000:
  _time_this_iter_s: 7.799881935119629
  _timestamp: 1655130855
  _training_iteration: 1
  date: 2022-06-13_16-34-15
  done: false
  experiment_id: e81f00ff591c41748d03b12cb54b6ed9
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 1
  loss: 2.1625621668092765
  node_ip: 127.0.0.1
  pid: 74893
  should_checkpoint: true
  time_since_restore: 11.236510038375854
  time_this_iter_s: 11.236510038375854
  time_total_s: 11.236510038375854
  timestamp: 1655130855
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: de45a_00000
  warmup_time: 0.0027399063110351562
  
Result for TorchTrainer_de45a_00000:
  _time_this_iter_s: 7.912680149078369
  _timestamp: 1655130863
  _training_iteration: 2
  date: 2022-06-13_16-34-23
  done: false
  experiment_id: e81f00ff591c41748d03b12cb54b6ed9
  hostname: Kais-MacBook-Pro.local
  iterations_since_restore: 2
  loss: 1.8909008730748655
  node_ip: 127.0.0.1
  pid: 74893
  should_checkpoint: true
  ti

2022-06-13 16:34:40,008	INFO tune.py:742 -- Total run time: 38.31 seconds (38.18 seconds for the tuning loop).


We can see our results here:

In [18]:
print(f"Last result: {result.metrics}")
print(f"Checkpoint: {result.checkpoint}")

Last result: {'loss': 1.2539631815472985, '_timestamp': 1655130879, '_time_this_iter_s': 7.77909517288208, '_training_iteration': 4, 'time_this_iter_s': 7.779023170471191, 'should_checkpoint': True, 'done': True, 'timesteps_total': None, 'episodes_total': None, 'training_iteration': 4, 'trial_id': 'de45a_00000', 'experiment_id': 'e81f00ff591c41748d03b12cb54b6ed9', 'date': '2022-06-13_16-34-39', 'timestamp': 1655130879, 'time_total_s': 34.665777921676636, 'pid': 74893, 'hostname': 'Kais-MacBook-Pro.local', 'node_ip': '127.0.0.1', 'config': {}, 'time_since_restore': 34.665777921676636, 'timesteps_since_restore': 0, 'iterations_since_restore': 4, 'warmup_time': 0.0027399063110351562, 'experiment_tag': '0'}
Checkpoint: <ray.air.checkpoint.Checkpoint object at 0x1b97a6090>


## Loading the model for prediction
You may have noticed that we skipped one part of the original tutorial - loading the model and using it for inference. The original code looks like this (we've wrapped it in a function):

In [19]:
def predict_from_model(model):
    classes = [
        "T-shirt/top",
        "Trouser",
        "Pullover",
        "Dress",
        "Coat",
        "Sandal",
        "Shirt",
        "Sneaker",
        "Bag",
        "Ankle boot",
    ]

    model.eval()
    x, y = test_data[0][0], test_data[0][1]
    with torch.no_grad():
        pred = model(x)
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}"')


We can use our saved model with the existing code to do prediction:

In [20]:
from ray.train.torch import load_checkpoint

model, _ = load_checkpoint(result.checkpoint, NeuralNetwork())

predict_from_model(model)

Predicted: "Ankle boot", Actual: "Ankle boot"


To predict more than one example, we can use a loop:

In [21]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

def predict_from_model(model, data):
    model.eval()
    with torch.no_grad():
        for x, y in data:
            pred = model(x)
            predicted, actual = classes[pred[0].argmax(0)], classes[y]
            print(f'Predicted: "{predicted}", Actual: "{actual}"')


In [22]:
predict_from_model(model, [test_data[i] for i in range(10)])

Predicted: "Ankle boot", Actual: "Ankle boot"
Predicted: "Pullover", Actual: "Pullover"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Pullover", Actual: "Shirt"
Predicted: "Trouser", Actual: "Trouser"
Predicted: "Coat", Actual: "Coat"
Predicted: "Coat", Actual: "Shirt"
Predicted: "Sneaker", Actual: "Sandal"
Predicted: "Sneaker", Actual: "Sneaker"


## Using Ray AIR for scalable batch prediction
However, we can also use Ray AIRs `BatchPredictor` class to do scalable prediction.

In [23]:
from ray.air import BatchPredictor
from ray.air.predictors.integrations.torch import TorchPredictor

batch_predictor = BatchPredictor.from_checkpoint(result.checkpoint, TorchPredictor, model=NeuralNetwork())

The Batch predictors work with Ray Datasets. Here we convert our test dataset into a Ray Dataset - note that this is not very efficient, and you can look at our [other tutorials](https://docs.ray.io/en/master/ray-air/examples/index.html) to see more efficient ways to generate a Ray Dataset.

In [24]:
import ray.data

ds = ray.data.from_items([x for x, y in test_data])

We can then trigger prediction with two workers:

In [25]:
results = batch_predictor.predict(ds, min_scoring_workers=2)

Map Progress (2 actors 1 pending): 100%|██████████████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 75.07it/s]


`results` is another Ray Dataset. We can use `results.to_pandas()` to see our prediction results:

In [26]:
results.to_pandas()

,predictions
0,"[-1.3483872, -1.9031825, -0.6305641, -1.572866..."
1,"[0.63100976, -2.0416622, 2.769512, -0.9032744,..."
2,"[1.6046226, 3.1562424, -0.16837832, 2.441095, ..."
3,"[1.1546009, 2.4993768, -0.20264417, 1.8872095,..."
4,"[0.71698636, -0.8087102, 1.0817742, -0.1187593..."
...,...
9995,"[-1.7012088, -2.886138, -0.6332233, -2.283086,..."
9996,"[0.804242, 1.57137, -0.0862837, 1.1939533, 0.0..."
9997,"[0.6843238, -0.0022547953, -0.103323564, 0.518..."
9998,"[1.1295415, 2.4181657, -0.3236364, 1.9868383, ..."


If we want to convert these predictions into class names (as in the original example), we can use a `map` function to do this:

In [27]:
predicted_classes = results.map_batches(
    lambda batch: [classes[pred.argmax(0)] for pred in batch["predictions"]], 
    batch_format="pandas")

Map_Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 215.34it/s]


To compare this with the actual labels, let's create a Ray dataset for these and zip it together with the predicted classes:

In [28]:
real_classes = ray.data.from_items([classes[y] for x, y in test_data])
merged = predicted_classes.zip(real_classes)

Let's examine our results:

In [29]:
merged.to_pandas()

,value
0,"(Ankle boot, Ankle boot)"
1,"(Pullover, Pullover)"
2,"(Trouser, Trouser)"
3,"(Trouser, Trouser)"
4,"(Pullover, Shirt)"
...,...
9995,"(Ankle boot, Ankle boot)"
9996,"(Trouser, Trouser)"
9997,"(T-shirt/top, Bag)"
9998,"(Trouser, Trouser)"


## Summary

This tutorial demonstrated how to turn your existing PyTorch code into code you can use with Ray AIR.

We learned how to
- enable distributed training using Ray Train abstractions
- save and retrieve model checkpoints via Ray AIR
- load a model for batch prediction

In our [other examples](https://docs.ray.io/en/master/ray-air/examples/index.html) you can learn how to do more things with the Ray AIR API, such as **serving your model with Ray Serve** or **tune your hyperparameters with Ray Tune.** You can also learn how to **construct Ray Datasets** to leverage Ray AIR's **preprocessing** API.

We hope this tutorial gave you a good starting point to leverage Ray AIR. If you have any questions, suggestions, or run into any problems pelase reach out on [Discuss](https://discuss.ray.io/) or [GitHub](https://github.com/ray-project/ray)!